# LangChain Expression Language (LCEL)

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
#!pip install pydantic==1.10.8

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [6]:
prompt = ChatPromptTemplate.from_template(
    "tell me a fun fact about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [7]:
chain = prompt | model | output_parser

In [10]:
chain.invoke({"topic": "space"})

'One fun fact about space is that the International Space Station travels at a speed of approximately 28,000 kilometers per hour (17,500 miles per hour) and orbits the Earth about every 90 minutes. This means that astronauts on board experience 16 sunrises and sunsets every day!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [11]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [13]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["abdullah worked at centrox ai", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [14]:
retriever.get_relevant_documents("where did abdullah work?")

/home/centrox_ai/anaconda3/envs/llava/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='abdullah worked at centrox ai'),
 Document(page_content='bears like to eat honey')]

In [15]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='abdullah worked at centrox ai')]

In [16]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [17]:
from langchain.schema.runnable import RunnableMap

In [18]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [19]:
chain.invoke({"question": "where did abdullah work?"})

'Abdullah worked at Centrox AI.'

In [20]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [21]:
inputs.invoke({"question": "where did abdullah work?"})

{'context': [Document(page_content='abdullah worked at centrox ai'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did abdullah work?'}

## Bind

and OpenAI Functions

In [22]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [24]:
runnable = prompt | model

In [25]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': '{"airport_code":"SFO"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x7de8d326ad50> JSON: {
  "prompt_tokens": 64,
  "completion_tokens": 16,
  "total_tokens": 80
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_a450710239', 'finish_reason': 'function_call', 'logprobs': None}, id='run-f71b35a2-8cfd-4570-b454-e43211ea91d8-0')

In [26]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [27]:
model = model.bind(functions=functions)

In [28]:
runnable = prompt | model

In [29]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': '{"team_name":"New England Patriots"}'}}, response_metadata={'token_usage': <OpenAIObject at 0x7de8d3269370> JSON: {
  "prompt_tokens": 99,
  "completion_tokens": 18,
  "total_tokens": 117
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-45898f8e-18c2-40c8-aa12-89d698e9d62d-0')

## Fallbacks

In [30]:
from langchain.llms import OpenAI
import json

In [31]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="text-davinci-001"
)
simple_chain = simple_model | json.loads

/home/centrox_ai/anaconda3/envs/llava/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [32]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [33]:
simple_model.invoke(challenge)

InvalidRequestError: The model `text-davinci-001` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations

Note: The next line is expected to fail.

In [34]:
simple_chain.invoke(challenge)

InvalidRequestError: The model `text-davinci-001` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations

In [35]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [31]:
chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [36]:
final_chain = simple_chain.with_fallbacks([chain])

In [37]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose is a rose is a rose'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers'}}

## Interface

In [38]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [39]:
chain.invoke({"topic": "bears"})

'Why did the bear dissolve in water?\n\nBecause it was a polar bear!'

In [40]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why did the bear wear a hat to the party? \nBecause he didn't want to look bear-headed!",
 'Why are frogs so happy?\nBecause they eat whatever bugs them!']

In [41]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 do
 bears
 have
 hairy
 coats
?


F
ur
 protection
!



In [42]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!"